In [1]:
import re
import math
import sys
import random

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 269
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1313 words
filter frequent words: 540 words
filter letter words: 539 words
filter stop words: 352 words


In [4]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### SDC

In [6]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(doc))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def sdc(bow_corpus, min_samples, eps):
    delta_eps = eps / 10
    labels = [-1 for i in range(len(bow_corpus))]
    sims = cosine_similarity(bow_corpus)
    
    points = [i for i in range(len(bow_corpus))]
    cluster_num = 0
    while len(points) > 0:
        seed = random.choice(points)
        eps_neighbors = [i for i, sim in enumerate(sims[seed]) if sim >= eps and labels[i] == -1]
        if len(eps_neighbors) >= min_samples:
            cluster_num += 1
            for p in eps_neighbors:
                labels[p] = cluster_num
            points = [i for i in points if i not in eps_neighbors]
        else:
            labels[seed] = 0
            points.remove(seed)

    while cluster_num != 0:
        cluster = [numpy.array(bow_corpus.iloc[i]) for i, label in enumerate(labels) if label == cluster_num]
        eps_temp = eps
        
        while True:
            centroid = numpy.mean(cluster, axis=0).reshape(1, -1)
            eps_temp -= delta_eps
            
            count = 0
            for i, label in enumerate(labels):
                point = numpy.array(bow_corpus.iloc[i]).reshape(1, -1)
                if label == 0 and cosine_similarity(centroid, point) >= eps_temp:
                    cluster.append(point[0])
                    labels[i] = cluster_num
                    count += 1
            if count == 0:
                break
        
        cluster_num -= 1
    
    return labels

In [8]:
def upgrade_sdc(bow_corpus, min_samples, eps):
    labels = [-1 for i in range(len(bow_corpus))]
    initials = []
    delta_eps = eps / 10
    sims = cosine_similarity(bow_corpus)
    
    points = [i for i in range(len(bow_corpus))]
    clusters = []
    cluster_num = 0
    clusters.append([])
    while len(points) > 0:
        seed = random.choice(points)
        eps_neighbors = [i for i, sim in enumerate(sims[seed]) if sim >= eps and labels[i] == -1]
        if len(eps_neighbors) >= min_samples:
            print(seed)
            cluster_num += 1
            clusters.append([])
            for p in eps_neighbors:
                initials.append(p)
                labels[p] = cluster_num
                clusters[cluster_num].append(numpy.array(bow_corpus.iloc[p]))
            points = [i for i in points if i not in eps_neighbors]
        else:
            labels[seed] = 0
            clusters[0].append((seed, numpy.array(bow_corpus.iloc[seed])))
            points.remove(seed)
            
    expandable = numpy.zeros(cluster_num + 1)    
    while sum(expandable) != -cluster_num - 1:
        eps -= delta_eps
        count = numpy.zeros(cluster_num + 1)
        for point in clusters[0]:
            if labels[point[0]] != 0:
                continue

            num = point[0]
            p = point[1].reshape(1, -1)
            sim = 0
            for c, cluster in enumerate(clusters[1:]):
                if expandable[c + 1] == -1:
                    continue
                centroid = numpy.mean(cluster, axis=0).reshape(1, -1)
                if cosine_similarity(centroid, p) >= sim:
                    sim = cosine_similarity(centroid, p)
                    if sim >= eps:
                        labels[num] = c + 1

            if labels[num] != 0:
                count[labels[num]] += 1
                clusters[labels[num]].append(point[1])

        for i, num in enumerate(count):
            if num == 0:
                expandable[i] = -1
        
    return labels, initials

In [9]:
def predict_cluster(bow_corpus, min_samples, eps):
    predicted_labels, initials = upgrade_sdc(bow_corpus, min_samples, eps)

    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    
    return result, initials

In [10]:
# bow_corpus = get_bow(idx_corpus)
bow_corpus = get_bow(filtered_corpus)
result, initials = predict_cluster(bow_corpus, 7, 0.3)
bow_corpus.head()

25
265
153
200
134
198
85
77
154
82
41
184


,ทำ,พนักงาน,ข้าว,ดี,น้ำ,อี,เครื่อง,เลิก,กก,งง,...,พลังงาน,คอย,แอร์ไม่,สลิป,คู,ปอง,นวมินทร์,สิบ,ห้า,แจก
0,0.333333,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Result

In [11]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_clusters = label_count[0][-1] + 1
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]), array([59, 52, 31, 21, 13, 17, 15, 14, 12,  9,  8,  7, 11]))
		percent
0  |	59	1.0


In [12]:
comment_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        if index in initials:
            print("*", end="")
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=1)

1 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว
5 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
20 เช็คเอาท์  หน้าจะงอไปไหนสวัสดีค่ะมีอะไรให้ช่วยมั้ยคะ
26 โลตลาด พนงปากตลาด นินทาลคเผาขน ตะโกนโหวกเหวกข้ามหัวลค วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ
29 ไม่ต้องตัดแต้มเป็นคูปองเงินได้ไหมอ่ะ ไม่เคยได้ใช้เพราะ คูปองมันหายก่อน เอาแบบ พอไปซื้อแล้วแต้มครบค่อยถามว่าตัดแต้มไหมคะ อะไรแบบนี้จะดีกว่า  แล้วยังมีโบชัวมาบอกอีกนะ แต่ม เท่านี้แล้วเป็นตั๋วหนังได้ แล้วเล่นตัดไปแล้วมันเหลือแต้ม150จะทำไรได้
31 ตะคิดเป็น 25 สต 50สต 75สต ทำไม ในเมื่อลวท้ายด้วยราคาพวกนี้ ตอนคิดเงินปัดเศษขึ้นไปเต็มบาทเลยพอเราเอาเหรียญสตางค์ที่เป็นเศษให้ ชักสีหน้า ไม่อยากรับ
32 สาขายโสธรเปิดใหม่ พนงแอบโกง   ลูกค้ากุเอง จ่ายหลายใบแล้วทำเนียนว่ามีใบนึงชำรุดใช้ไม่ได้แล้วพอขอคืนก็สลับใบที่รูดใช้แล้วมาแทน แจ้งผู้จัดการขอดูกล้องวงจรปิดไม่ยอมให้ดูเลยต้องขู่ว่าจะแจ้งความถึงยอมคืนให้ แต่ไม่มีคำขอโทษใดๆ จาก พนงติด
43 มารยาท คำพูดคำจา ไม่ต้องเพราะหรอกคะแค่ไม่ตะคอก ไม่ชักสี

In [13]:
token_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=1)

1 ['ดี', 'เลิก', 'น้ำ', 'เครื่อง', 'ข้าว', 'อี']
5 ['อย่า', 'อย่า', 'กี่', 'ปี', 'บังคับ', 'น้อง', 'เชียร์', 'งี่', 'เง่า']
20 ['หน้า', 'เช็ค', 'งอ', 'สวัสดี']
26 ['ปรับปรุง', 'พนง', 'แย่', 'ตะโกน', 'ข้าม', 'หวย', 'พิเศษ', 'พฤติกรรม']
29 ['เงิน', 'ซื้อ', 'ดี', 'ถาม', 'อ่ะ', 'หาย', 'ไหม', 'ไหม', 'เล่น']
31 ['เงิน', 'ราคา', 'ตอน', 'สีหน้า', 'เต็ม', 'บาท', 'เศษ', 'สตางค์', 'เศษ']
32 ['ลูกค้า', 'จ่าย', 'พนง', 'พนง', 'ใบ', 'ใบ', 'ใบ', 'ติด', 'นึง']
43 ['เวลา', 'เวลา', 'บริการ', 'สินค้า', 'ถาม', 'มารยาท', 'เจอ', 'ตอน', 'บ้าน']
57 ['รู้', 'ผัก', 'เน่า', 'เน่า', 'ทิ้ง', 'ป้ายเหลือง']
59 ['ห้าง', 'อาหาร', 'กลิ่น', 'ฝุ่น']
69 ['เงิน', 'สินค้า', 'ราคา', 'ราคา', 'ตอน', 'แคชเชียร์', 'วาง', 'ชั้น']
71 ['ซื้อ', 'เวลา', 'ผม', 'กะ', 'อย่า', 'วาง', 'มือ', 'เย็น', 'ล้าง']
84 ['สาขา', 'ตอน', 'เลิก', 'ติด', 'ผม', 'กลิ่น', 'เหม็น', 'ฝาก', 'เช้า']
91 ['เดิน', 'รู้', 'อาหาร', 'เหม็น', 'ข้าว', 'หนู', 'หนู', 'โซน', 'แห้ง']
95 ['สาขา', 'เย็น', 'ตู้', 'ตู้', 'ตู้']
98 ['เงิน', 'บริการ', 'สินค้า', 'พนง', 'ร้าน', '

In [14]:
seed = 164
compare = 91

a = numpy.array(bow_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(bow_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(filtered_corpus[compare])

[[0.17407766]]
['แย่', 'อาหาร', 'แถม']
['เดิน', 'รู้', 'อาหาร', 'เหม็น', 'ข้าว', 'หนู', 'หนู', 'โซน', 'แห้ง']
